In [1]:
import os
import torch
import numpy as np
import pandas as pd

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

from io import open
import glob
import os
import string

In [20]:

all_letters = string.ascii_letters[:26]
all_letters.append('_')
n_letters = len(all_letters)

n_categories = 26 #number of letters in english language

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor


torch.tensor([all_letters.index('c')],dtype=torch.long)

tensor([2])

In [11]:
file_path = "C:/Users/cowpo/Downloads/dataset.txt"
df = pd.read_csv(file_path, header = None)

words = df[1]
input = df[0]

In [4]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size) #may be vanishing or exploding
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden): #this hidden parameter is h_{t-1}
        hidden = F.tanh(self.i2h(input) + self.h2h(hidden)) #updates hidden layer to use for next step
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size) # 1 * hidden_size matrix of zeroes. Set up this way so matrix multiplication is easier

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [5]:
#example input
input = letterToTensor('p')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden) #output has probabilies. This is applying the forward method


In [6]:
#decides our guess

def letterFromOutput(output):
    top_n, top_i = output.topk(1) #value and index of our guess
    letter_i = top_i[0].item() #actual index
    return all_letters[letter_i], letter_i

print(letterFromOutput(output)) #example



('r', 17)


In [30]:
import random

def randomChoice(l):
    return random.randint(0, len(l) - 1)

def randomTrainingExample():
    rand_index= randomChoice(words)
    line = input[rand_index]
    word = words[rand_index] 
    word_length= len(word)


    missing = set()
    for i in range(word_length):
        if line[i] == '_':
            missing.add(word[i])
    
    probability = 1/len(missing)
    answer = [0 for _ in range(n_categories)]
    for let in missing:
        answer[all_letters.index(let)] = probability
    answer_tensor = torch.tensor([answer])
    line_tensor = lineToTensor(line)
    return line, word, line_tensor,answer_tensor

for i in range(10):
    line, word, line_tensor,answer_tensor = randomTrainingExample()
    print('input =', line, '/ word =', word, answer_tensor)

input = __r_hoi___ / word = serphoidea tensor([[0.2000, 0.0000, 0.0000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2000, 0.0000, 0.0000,
         0.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])
input = _r____tic / word = prakritic tensor([[0.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2000,
         0.0000, 0.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2000, 0.0000, 0.2000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])
input = quis_ / word = quist tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0.]])
input = _asq___aded / word = masqueraded tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000, 0.2500,
         0.0000, 0.0000, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])
input = __sur_ac___

In [8]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.005

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad() # set all inital graidents to zero

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()


In [ ]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print ``iter`` number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0